<a href="https://colab.research.google.com/github/pavan581/earth-git/blob/master/beta_veg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
# Impotr and initialize Earth Engine
import ee

ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWj1uH_4K19gMuxMUKqG0W9s5Y_s0feAmJCb23jC5n--Uk75x-tEXNI



Successfully saved authorization token.


In [2]:
# GeoJSON to work with GeoJson Files
import geojson

# Libraries to plot maps
import matplotlib.pyplot as plt
from IPython.display import Image

import folium
from ipyleaflet import GeoJSON
import geemap

%matplotlib inline

#### Run below cell in Colab Only

In [ ]:
!pip install geojson
!pip install geemap

# Git Repository of the Project
!git clone https://github.com/pavan581/earth-git.git

# Data and Feature Import

In [3]:
with open('2011_Dist.json') as f:
    data = geojson.load(f)

In [4]:
for dist in data['features']:
    if dist["properties"]["DISTRICT"] == 'East Godavari':
        aoi_data = dist
        break

aoi_coords = aoi_data["geometry"]["coordinates"]
aoi = ee.Geometry.Polygon(aoi_coords)

sentinel2_data = ee.ImageCollection("COPERNICUS/S2_SR")

In [5]:
img = ee.Image(
    sentinel2_data
    .filterBounds(aoi)
    .filterDate(ee.Date('2022-01-01'), ee.Date('2022-03-10'))
    .first()
    .clip(aoi)
)

In [29]:
url = img.select(['B4', 'B3', 'B2']).getThumbURL({'min':0, 'max':4000})
Image(url=url, width=500)

In [7]:
#aoi = ee.FeatureCollection(data).filter("DISTRICT == 'EAST GODAVARI'")

In [6]:
Map = geemap.Map()  # Add Google Satellite
Map.setCenter(82.5, 17.2, zoom=8.5)
Map

Map(center=[17.2, 82.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

In [25]:
Map.addLayer(img, vizParams)

In [ ]:


url = alpha_img.select('B1').getThumbURL({'min':0, 'max':2000})
Image(url=url, width=500)

In [24]:
#Define the visualization parameters.
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0.0,
  'max': 255
}

# Testing...

## NDVI

### Formula

In [7]:
ndviParams = {'min': -0.5, 'max': 1, 'palette': ['yellow', 'white', 'green']}

In [9]:
# Formula
nir = img.select("B8")
red = img.select("B4")
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')

Map.addLayer(ndvi, ndviParams, 'NDVI image')

In [16]:
# Method
ndvi_met = img.normalizedDifference(['B8', 'B4'])
Map.addLayer(ndvi_met, ndviParams, 'NDVI')

## NDWI

In [18]:
waterPalette = ['red', 'yellow', 'green', 'blue']
ndwiParams = {'min':-1, 'max'}
ndwi = img.normalizedDifference(['B3', 'B8'])
Map.addLayer(ndwi, {'min': -1, 'max': 0.5, 'palette': waterPalette}, 'NDWI');
Map

Map(bottom=59477.0, center=[17.2, 82.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widge…